In [6]:
# import library

import os, os.path
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
import csv
from scipy.stats import kurtosis, skew
import glob
from sklearn.preprocessing import MinMaxScaler
from skimage.feature import greycomatrix, greycoprops
from sklearn.cluster import KMeans
from collections import Counter
import math
import pandas as pd

In [17]:
# kernel = np.ones((5,5),np.uint8)
# scaler = MinMaxScaler()

In [10]:
# mengambil label gambar berdasar foldernya
labels = [os.path.dirname(file).split('\\')[1] for file in glob.glob("./downloads/*/*")]

In [19]:
# file_name = [[os.path.dirname(file).split('\\')[1], os.path.basename(file)] for file in glob.glob("./downloads/*/*")]
# new = [x for x in file_name]

# file_name

In [11]:
# read image

images = [cv2.imread(file) for file in glob.glob("./downloads/*/*")]

In [12]:
# fungsi resize citra
def resize_image(img):
    scale_percent = 50 # percent of original size
    width = int(img.shape[1] * scale_percent / 100)
    height = int(img.shape[0] * scale_percent / 100)
    dim = (width, height)
    # resize image
    resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    
    return resized

In [13]:
# fungsi ekstrak fitur color moment
def get_color_feature(img):
    means = np.mean(img)
    stdev = np.std(img)
    skewness = [skew(x) for x in img]
    kurt = [kurtosis(x) for x in img]

    return [means, stdev, skew(skew(skewness)), kurtosis(kurtosis(kurt))]

In [16]:
# fungsi kalkulasi matriks glcm dengan sudut gambar 0, 45, 90, 135 derajat

def calc_glcm_all_agls(img, label, props, dists=[5], agls=[0], lvl=256, sym=True, norm=True):
    
    glcm = greycomatrix(img, 
                        distances=dists, 
                        angles=agls, 
                        levels=lvl,
                        symmetric=sym, 
                        normed=norm)
    feature = []
    glcm_props = [propery for name in props for propery in greycoprops(glcm, name)[0]]
    for item in glcm_props:
            feature.append(item)
    feature.append(label) 
    
    return feature

In [19]:
# fungsi get glcm

def glcm_features(img, label="no label"):
    properties = ['dissimilarity', 'correlation', 'homogeneity', 'contrast', 'ASM', 'energy']
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    glcm_feat = calc_glcm_all_agls(gray, label, props=properties)
        
    return glcm_feat

In [28]:

# ekstraksi fitur dari citra
for image, label in zip(images, labels):
    glcm_feature = glcm_features(image, label)

    # Gaussian filtering
    blur = cv2.GaussianBlur(image,(5,5),0)
    color_feature = get_color_feature(blur)
    imgs_feat_A.append(color_feature + glcm_feature)

In [30]:
# menyimpan feature ke dalam file csv
f = open('image_feature.csv', 'w', newline='\n', encoding='utf8')

with f:
    writer = csv.writer(f)
    writer.writerows(imgs_feat_A)

In [70]:
# membaca nilai fitur dari file csv (tidak perlu perhitungan fitur lagi)
df = pd.read_csv('image_feature.csv')

img_features = []

for i in range(0, 1499):
    temp = []
    for x in df:
        temp.append(df[x][i])
        
    img_features.append(temp)

In [54]:
# pembulatan nilai feature
rounded_scale = [[round(x, 5) for x in y[0:9]] for y in img_features]
# rounded_scale

In [62]:
# clustering k-means dengan jumlah cluster 50 dan iterasi maksimum 200
kmeans = KMeans(n_clusters=25, max_iter=200).fit(rounded_scale)

y = kmeans.predict(rounded_scale)

In [63]:
# menyatukan feature dengan label yang sesuai

with_nama_makanan = []
for i in range(len(rounded_scale)):
    label = [labels[i]]
    with_nama_makanan.append(rounded_scale[i] + label)

In [64]:
# menyatukan citra dengan cluster hasil clustering k-means
with_label = list(zip(with_nama_makanan, y))

# mengurutkan hasil clustering
temp = sorted(with_label, key=lambda x: x[1])

# inisialisasi variabel baru untuk menyimpan data berdasar cluster
new = []

# pemisahan data untuk setiap cluster
for i in range(0, 50):
    temps = []
    for x in temp:
        if i == x[1]:
            temps.append(x[0])
    new.append(temps)

In [65]:
# centroid dari hasil k-means clustering
centroids = kmeans.cluster_centers_

In [66]:
# fungsi euclidean distance
def euclidean_distance(input_data, dataset):
    distances = []
    for data in dataset:
        dist = math.sqrt(sum([(a - b) ** 2 for a, b in zip(input_data, data)]))
        distances.append(dist)
    
    return distances

In [67]:
# fungsi klasifikasi knn
def knn(k, datatest, datatrain, nearest_cluster):
    data = datatrain[nearest_cluster]
    
    data_without_label = [x[0:9] for x in data]
    label_only = [x[-1] for x in data]
    
    distance = euclidean_distance(datatest, data_without_label)
    zipped = list(zip(distance, label_only))
    temp = sorted(zipped, key=lambda x: x[0])
    
    nearest_neighbor = temp[:k]
    
    most_label = [x[1] for x in nearest_neighbor]
    
    return Counter(most_label).most_common(1)[0][0]

In [69]:
# image untuk testing
test_images = [cv2.imread(file) for file in glob.glob("./sampel gambar/*")]

# inisialisasi variabel untuk menyimpan fitur citra testing
test_feat = []

for test in test_images:

    resized_imagetest = resize_image(test)

    # Gaussian filtering
    blurtest = cv2.GaussianBlur(resized_imagetest,(5,5),0)
    test_feat.append(get_color_feature(blurtest) + glcm_features(resized_imagetest))

for x in test_feat:
    cluster_distance_test = euclidean_distance(x, centroids)
    nearest_cluster_test = cluster_distance_test.index(min(cluster_distance_test))
#     print(nearest_cluster_test)
    print(knn(15, x, new, nearest_cluster_test))

Mie Ayam
sate
sate
